<a href="https://colab.research.google.com/github/SantosCristiano/artificial-intelligence-python/blob/main/Retrieval_Augmented_Generation_(RAG)_Using_Mistral_AI_And_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Passo 1
Vamos começar com a implementação do código, primeiro você precisará instalar todas as bibliotecas Python necessárias para o código.

In [ ]:
!pip install chromadb
!pip install langchain
!pip install pypdf
!pip install sentencepiece
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/UKPLab/sentence-transformers.git
!pip install git+https://github.com/Muennighoff/sentence-transformers.git@sgpt_poolings_specb
!pip install --upgrade git+https://github.com/UKPLab/sentence-transformers.git
!pip install -U sentence-transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/UKPLab/sentence-transformers.git to /tmp/pip-req-build-x6t8om26
  Running command git clone --filter=blob:none --quiet https://github.com/UKPLab/sentence-transformers.git /tmp/pip-req-build-x6t8om26
  Resolved https://github.com/UKPLab/sentence-transformers.git to commit 579257a3500380f6484ece4c84587500bc427acc
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/Muennighoff/sentence-transformers.git (to revision sgpt_poolings_specb) to /tmp/pip-req-build-kxw8d_qp
  Running command git clone --filter=blob:none --quiet https://github.com/Muenni

# Passo 2
Agora, iniciaremos a configuração do Modelo de Linguagem (**LLM**) tanto para a criação de **embeddings** de texto quanto para a geração de respostas. Utilizaremos o modelo “**all-mpnet-base-v2**” para os **embeddings**. Para a geração de respostas, empregaremos o modelo “**Mistral-7B-Instruct-v0.2**”, mais especificamente, sua versão quantizada. A carga desse modelo quantizado exige uma **GPU** com, no mínimo, **16 GB de RAM**. Verifique se sua **GPU** cumpre esse requisito. Precisaremos de uma configuração capaz de carregar o modelo em **4 bits**, conforme a quantização **FP4**. Para tal fim, utilizamos uma conta **Colab Pro** que dispõe de uma **GPU T4**.

No código a seguir, importamos as bibliotecas necessárias, realizamos a técnica de quantização e, depois, carregamos o modelo quantizado usando o Pipeline do HuggingFace.

In [ ]:
# carrega a biblioteca necessária
import os
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

quantization_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_compute_dtype=torch.bfloat16
)

model_kwargs = {'device': 'cuda'}
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=model_kwargs)
embeddings = HuggingFaceEmbeddings(model_name="distilbert-base-uncased", model_kwargs=model_kwargs)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", device_map='auto', quantization_config=quantization_config)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Passo 3
Prosseguiremos lendo os dados do **PDF** usando o **PyPDFLoader** da **Langchain**. Você precisa fornecer o link da web onde seu **PDF** está hospedado ou o caminho local do seu sistema onde o **PDF** está localizado.

Tô usando a **Constituição da República Federativa do Brasil**. Você pode baixar o PDF, colocá-lo em seu diretório de trabalho atual e fornecer seu caminho para a variável chamada “pdf_link” no código abaixo:
[Constituição Federal](https://www.stf.jus.br/arquivo/cms/legislacaoConstituicao/anexo/CF.pdf)

Assim que os dados do PDF forem carregados, iremos processá-los em partes usando o “**RecursiveCharacterTextSplitter**” do **Langchain**. Esta ferramenta pegará os dados e os dividirá em partes gerenciáveis para processamento posterior.

In [ ]:
# Carrega o arquivo PDF
pdf_link = "/content/sample_data/CF.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()


# Divide os dados em pedaços(chunks)
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size = 4000,
   chunk_overlap  = 20,
   length_function = len,
   add_start_index = True,
)
chunks = text_splitter.split_documents(pages)

# Passo 4
Depois de ingerirmos e transformarmos os dados com sucesso, a próxima etapa envolve armazená-los no banco de dados Chroma. Para isso, forneceremos os blocos de dados que queremos armazenar no banco de dados, junto com o nome do modelo de incorporação. O sistema criará internamente incorporações dos dados de texto dos pedaços e os armazenará no banco de dados. O nome do banco de dados neste exemplo é **test_index**, mas fique à vontade para alterá-lo de acordo com suas preferências.

In [ ]:
# Armazena os dados no banco de dados
db=Chroma.from_documents(chunks, embedding=embeddings, persist_directory="test_index")
db.persist()

# Passo 5
Depois que os dados forem armazenados com sucesso no banco de dados, não há necessidade de repetir as etapas anteriores todas as vezes. Você pode simplesmente carregar o banco de dados pré-carregado conforme descrito nas linhas de código a seguir.

Em seguida, inicializaremos o recuperador, que se encarrega de recuperar do banco de dados o pedaço mais adequado que possa conter a resposta à pergunta do usuário. “**search_kwargs**” com “**k**” definido como 3 neste contexto significa que ele recuperará os três principais pedaços mais relevantes do banco de dados.

Você pode alterar o prompt “**qna_prompt_template**” de acordo com seus requisitos.

A seguir, carregaremos uma cadeia QNA, que envolve o uso do modelo LLM para gerar uma resposta, além de especificar o tipo de cadeia.

In [ ]:
# Carrega o banco de dados de vetores
vectordb = Chroma(persist_directory="test_index", embedding_function=embeddings)

# Carrega o recuperador
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Template de prompt para Pergunta e Resposta
qna_prompt_template = """### [INST] Instrução: Serão fornecidas perguntas e dados relacionados. Sua tarefa é encontrar as respostas para as perguntas usando os dados fornecidos. Se os dados não contiverem a resposta para a pergunta, então você deve retornar 'Informação insuficiente.'

{context}

### Pergunta: {question} [/INST]"""

# Cria o template de prompt
PROMPT = PromptTemplate(
   template=qna_prompt_template, input_variables=["context", "question"]
)

# Carrega a cadeia de pergunta e resposta
chain = load_qa_chain(llm, chain_type="stuff", prompt=PROMPT)

# Passo 6
No passo seguinte, definiremos uma função auxiliar que irá gerar uma resposta. Esta função terá como entrada a pergunta do usuário. Passaremos a consulta do usuário para o recuperador usando esta função. Por sua vez, o recuperador irá combinar internamente a incorporação da questão com os documentos armazenados no banco de dados e recuperar o pedaço mais apropriado. Esse pedaço, junto com a pergunta, será então passado para a cadeia QNA, que gerará a resposta.

In [ ]:
def ask(question):
    max_length = 200
    aviso_fora_de_contexto = "Aviso: A pergunta pode estar fora do contexto dos documentos fornecidos. Fornecendo uma resposta com base no conhecimento geral do modelo."
    prompt = "Responda sempre em português: " + question

    # Tenta encontrar documentos relevantes para a pergunta
    context = retriever.get_relevant_documents(question)

    try:
        if not context:
            # Caso não haja contexto relevante encontrado
            full_response = llm.invoke(prompt, max_length=max_length)
            response = aviso_fora_de_contexto + "\n\nResposta: " + full_response.strip().split("\n")[0]
        else:
            chain_response = chain.run({"input_documents": context, "question": question})
            if "Informação insuficiente" in chain_response or chain_response.strip() == "":
                # Caso o contexto seja insuficiente para responder à pergunta
                full_response = llm.invoke(prompt, max_length=max_length)
                response = aviso_fora_de_contexto + "\n\nResposta: " + full_response.strip().split("\n")[0]
            else:
                # Quando há contexto suficiente para responder à pergunta
                full_response = chain_response
                response = "Resposta: " + full_response.strip().split("\n")[0]

        if len(response) >= max_length - 1:
            # Procura o último ponto final antes do limite para uma conclusão lógica
            last_period = response.rfind('.')
            response = response[:last_period + 1] if last_period != -1 else response
            response += " [Resposta truncada. Consulte a fonte para mais detalhes.]"

    except Exception as e:
        response = f"Erro ao gerar resposta: {e}"

    return response


# Passo 7
Agora, com tudo pronto, estamos prontos para realizar perguntas e respostas (QnA) nos dados do PDF. Para fazer uma pergunta, basta adicionar as seguintes linhas de código ao seu script:

## Sugestões de perguntas


*   Quais são as responsabilidades dos municípios segundo a Constituição Brasileira?
*   Como a Constituição Brasileira aborda os direitos dos povos indígenas?
*   Como a Constituição Federal define a organização dos poderes no Brasil?
*   O que a Constituição Brasileira diz sobre a educação?
*   Qual é a importância do artigo 5º da Constituição Brasileira?




In [ ]:

# Pega a entrada do usuário e chama a função para gerar a saída
user_question = input("Usuário: ")
answer = ask(user_question)
print(answer)